In [1]:
import numpy as np
import pandas as pd

from tqdm import tqdm
import os


In [2]:
train_df = pd.read_csv('train.csv')


In [3]:
test_names, test_texts = [], []
for f in tqdm(list(os.listdir('./test'))):
    test_names.append(f.replace('.txt', ''))
    test_texts.append(open('./test/' + f, 'r').read().replace(u'\xa0', u' '))
test_texts = pd.DataFrame({'id': test_names, 'text': test_texts})
# test_texts['text'] = test_texts['text'].apply(lambda x:x.split())
test_texts.head()

100%|██████████| 5/5 [00:00<00:00, 2087.76it/s]


,id,text
0,DF920E0A7337,Have you ever asked more than one person for h...
1,0FB0700DAF44,"During a group project, have you ever asked a ..."
2,D46BCB48440A,"When people ask for advice,they sometimes talk..."
3,18409261F5C2,80% of Americans believe seeking multiple opin...
4,D72CB1C11673,Making choices in life can be very difficult. ...


In [4]:
test_names, train_texts = [], []
for f in tqdm(list(os.listdir('./train'))):
    test_names.append(f.replace('.txt', ''))
    train_texts.append(open('./train/' + f, 'r').read().replace(u'\xa0', u' '))
train_text_df = pd.DataFrame({'id': test_names, 'text': train_texts})
# train_texts['text'] = test_texts['text'].apply(lambda x:x.split())
train_text_df.head()

100%|██████████| 15594/15594 [00:01<00:00, 8173.10it/s]


,id,text
0,3321A3E87AD3,I do agree that some students would benefit fr...
1,DFEAEC512BAB,Should students design a summer project for sc...
2,2E4AFCD3987F,"Dear State Senator\n\n,\n\nIn the ruels of vot..."
3,EB6C2AF20BFE,People sometimes have a different opinion than...
4,A91A08E523D5,"Dear senator,\n\nAs you know the Electoral Col..."


In [5]:
all_entities = []
for i in tqdm(train_text_df.iterrows()):
    total = i[1]['text'].split().__len__()
    start = -1
    entities = []
    for j in train_df[train_df['id'] == i[1]['id']].iterrows():
        discourse = j[1]['discourse_type']
        list_ix = j[1]['predictionstring'].split(' ')
#         print(j[1]['predictionstring'],'###' ,len(list_ix))
        ent = [f"I-{discourse}" for ix in list_ix]
        ent[0] = f"B-{discourse}"
        ds = int(list_ix[0])
        de = int(list_ix[-1])
        if start < ds-1:
            ent_add = ['O' for ix in range(int(ds-1-start))]
            ent = ent_add + ent
#         print(len(entities))
#         print(ent, len(ent))
        entities.extend(ent)
#         print(len(entities))
        start = de
    if len(entities) < total:
        ent_add = ["O" for ix in range(total-len(entities))]
        entities += ent_add
    else:
        entities = entities[:total]
#     print(i[1]['id'],'@@@@@@@@' ,i[1]['text'].split(' ').__len__(), len(entities))
    all_entities.append(entities)
#     if len(all_entities) > 100:
#         break

15594it [01:40, 155.69it/s]


In [8]:
train_text_df['entities'] = all_entities

In [9]:
train_text_df.head()

,id,text,entities
0,3321A3E87AD3,I do agree that some students would benefit fr...,"[B-Lead, I-Lead, I-Lead, I-Lead, I-Lead, I-Lea..."
1,DFEAEC512BAB,Should students design a summer project for sc...,"[O, O, O, O, O, O, O, O, B-Position, I-Positio..."
2,2E4AFCD3987F,"Dear State Senator\n\n,\n\nIn the ruels of vot...","[O, O, O, O, B-Position, I-Position, I-Positio..."
3,EB6C2AF20BFE,People sometimes have a different opinion than...,"[B-Lead, I-Lead, I-Lead, I-Lead, I-Lead, I-Lea..."
4,A91A08E523D5,"Dear senator,\n\nAs you know the Electoral Col...","[O, O, B-Lead, I-Lead, I-Lead, I-Lead, I-Lead,..."


In [11]:
from transformers import RobertaTokenizerFast, RobertaForTokenClassification
from torch.utils.data import Dataset, DataLoader
import pdb
import torch
from sklearn.metrics import accuracy_score

In [12]:
config = {'model_name': 'roberta-base',
         'max_length': 512,
         'train_batch_size':8,
         'valid_batch_size':16,
         'epochs':3,
         'learning_rate':1e-05,
         'max_grad_norm':10,
         'device': 'cuda' if torch.cuda.is_available() else 'cpu'}

In [13]:
output_labels = ['O', 'B-Lead', 'I-Lead', 'B-Position', 'I-Position', 'B-Claim', 'I-Claim', 'B-Counterclaim', 'I-Counterclaim', 
          'B-Rebuttal', 'I-Rebuttal', 'B-Evidence', 'I-Evidence', 'B-Concluding Statement', 'I-Concluding Statement']

labels_to_ids = {v:k for k,v in enumerate(output_labels)}
ids_to_labels = {k:v for k,v in enumerate(output_labels)}

In [14]:
labels_to_ids

{'O': 0,
 'B-Lead': 1,
 'I-Lead': 2,
 'B-Position': 3,
 'I-Position': 4,
 'B-Claim': 5,
 'I-Claim': 6,
 'B-Counterclaim': 7,
 'I-Counterclaim': 8,
 'B-Rebuttal': 9,
 'I-Rebuttal': 10,
 'B-Evidence': 11,
 'I-Evidence': 12,
 'B-Concluding Statement': 13,
 'I-Concluding Statement': 14}

In [15]:
class dataset(Dataset):
  def __init__(self, dataframe, tokenizer, max_len):
        self.len = len(dataframe)
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len

  def __getitem__(self, index):
        # step 1: get the sentence and word labels 
        sentence = self.data.text[index]
        word_labels = self.data.entities[index]

        # step 2: use tokenizer to encode sentence (includes padding/truncation up to max length)
        # BertTokenizerFast provides a handy "return_offsets_mapping" functionality for individual tokens
        encoding = self.tokenizer(sentence,
#                              is_pretokenized=True, 
#                                   is_split_into_words=True,
                             return_offsets_mapping=True, 
                             padding='max_length', 
                             truncation=True, 
                             max_length=self.max_len)
        
        # step 3: create token labels only for first word pieces of each tokenized word
#         pdb.set_trace()
        labels = [labels_to_ids[label] for label in word_labels] 
        # code based on https://huggingface.co/transformers/custom_datasets.html#tok-ner
        # create an empty array of -100 of length max_length
        encoded_labels = np.ones(len(encoding["offset_mapping"]), dtype=int) * -100
#         print(len(sentence), len(labels))
        # set only labels whose first offset position is 0 and the second is not 0
        i = 0
        for idx, mapping in enumerate(encoding["offset_mapping"]):
#             print(idx)
            if mapping[0] != 0 and mapping[0] != encoding['offset_mapping'][idx-1][1]:
            # overwrite label
#             pdb.set_trace()
#             print(mapping)
#             print(encoded_labels.shape, len(labels), idx, i)
                try:
                    encoded_labels[idx] = labels[i]
                except:
                    pass
                i += 1
            else:
                if idx==1:
    #                 print(idx)
                    encoded_labels[idx] = labels[i]
                    i += 1
        # step 4: turn everything into PyTorch tensors
        item = {key: torch.as_tensor(val) for key, val in encoding.items()}
        item['labels'] = torch.as_tensor(encoded_labels)
        
        return item

  def __len__(self):
        return self.len

In [16]:
tokenizer = RobertaTokenizerFast.from_pretrained(config['model_name'])
model = RobertaForTokenClassification.from_pretrained(config['model_name'], num_labels=len(output_labels))

Downloading: 100%|██████████| 478M/478M [00:46<00:00, 10.8MB/s] 
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForTokenClassification: ['lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You 

In [18]:
data = train_text_df[['text', 'entities']]
train_size = 0.8
train_dataset = data.sample(frac=train_size,random_state=200)
test_dataset = data.drop(train_dataset.index).reset_index(drop=True)
train_dataset = train_dataset.reset_index(drop=True)

print("FULL Dataset: {}".format(data.shape))
print("TRAIN Dataset: {}".format(train_dataset.shape))
print("TEST Dataset: {}".format(test_dataset.shape))

training_set = dataset(train_dataset, tokenizer, config['max_length'])
testing_set = dataset(test_dataset, tokenizer, config['max_length'])

FULL Dataset: (15594, 2)
TRAIN Dataset: (12475, 2)
TEST Dataset: (3119, 2)


In [19]:
train_params = {'batch_size': config['train_batch_size'],
                'shuffle': True,
                'num_workers': 1,
                'pin_memory':True
                }

test_params = {'batch_size': config['valid_batch_size'],
                'shuffle': True,
                'num_workers': 1,
                'pin_memory':True
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

In [20]:
test_texts_set = dataset(test_texts, tokenizer, config['max_length'])
test_texts_loader = DataLoader(test_texts_set, **test_params)

In [21]:
## -100으로 레이블링 되는 게 좀 이상함
## 출력과정에서 -100은 레이블링을 제외해야 하는것으로 기억하는데 이부분에 대한 구현이 이상함
i=0
for token, label in zip(tokenizer.convert_ids_to_tokens(training_set[0]["input_ids"]), training_set[0]["labels"]):
    print(training_set[0]['offset_mapping'][i], '@@@')
    if label == -100:
        print('{}  {}'.format(token, label))
    else:
        print('{}  {}'.format(token, ids_to_labels[int(label)]))
    i+=1

tensor([0, 0]) @@@
<s>  -100
tensor([0, 6]) @@@
Phones  O
tensor([ 7, 10]) @@@
Ġand  O
tensor([11, 18]) @@@
ĠDriving  O
tensor([18, 19]) @@@
Ċ  -100
tensor([19, 20]) @@@
Ċ  -100
tensor([20, 25]) @@@
Every  -100
tensor([26, 30]) @@@
Ġyear  B-Lead
tensor([31, 35]) @@@
Ġmany  I-Lead
tensor([36, 42]) @@@
Ġpeople  I-Lead
tensor([43, 49]) @@@
Ġaround  I-Lead
tensor([50, 53]) @@@
Ġthe  I-Lead
tensor([54, 59]) @@@
Ġworld  I-Lead
tensor([60, 63]) @@@
Ġare  I-Lead
tensor([64, 69]) @@@
Ġdying  I-Lead
tensor([70, 77]) @@@
Ġbecause  I-Lead
tensor([78, 80]) @@@
Ġof  I-Lead
tensor([81, 85]) @@@
Ġcell  I-Lead
tensor([86, 91]) @@@
Ġphone  I-Lead
tensor([92, 95]) @@@
Ġuse  I-Lead
tensor([ 96, 101]) @@@
Ġwhile  I-Lead
tensor([102, 109]) @@@
Ġdriving  I-Lead
tensor([109, 110]) @@@
.  -100
tensor([111, 118]) @@@
ĠInstead  I-Lead
tensor([119, 121]) @@@
Ġof  I-Lead
tensor([122, 129]) @@@
Ġkeeping  I-Lead
tensor([130, 135]) @@@
Ġtheir  I-Lead
tensor([136, 140]) @@@
Ġeyes  I-Lead
tensor([141, 143]) @@@
Ġon  I-

In [22]:
device = config['device']

In [38]:
model.to(device)
inputs = training_set[0]
input_ids = inputs["input_ids"].unsqueeze(0)
attention_mask = inputs["attention_mask"].unsqueeze(0)
labels = inputs["labels"].unsqueeze(0)

input_ids = input_ids.to(device)
attention_mask = attention_mask.to(device)
labels = labels.to(device)

outputs = model(input_ids, attention_mask=attention_mask, labels=labels, return_dict=False)
initial_loss = outputs[0]
pred = outputs[1][0]
_, pred = torch.max(pred,1)
initial_loss

tensor(2.5425, grad_fn=<NllLossBackward>)

In [35]:
optimizer = torch.optim.Adam(params=model.parameters(), lr=config['learning_rate'])

In [ ]:
def train(epoch):
    tr_loss, tr_accuracy = 0, 0
    nb_tr_examples, nb_tr_steps = 0, 0
    tr_preds, tr_labels = [], []
    # put model in training mode
    model.train()
    
    for idx, batch in enumerate(training_loader):
        
        ids = batch['input_ids'].to(device, dtype = torch.long)
        mask = batch['attention_mask'].to(device, dtype = torch.long)
        labels = batch['labels'].to(device, dtype = torch.long)

        loss, tr_logits = model(input_ids=ids, attention_mask=mask, labels=labels, return_dict=False)
        tr_loss += loss.item()

        nb_tr_steps += 1
        nb_tr_examples += labels.size(0)
        
        if idx % 100==0:
            loss_step = tr_loss/nb_tr_steps
            print(f"Training loss per 100 training steps: {loss_step}")
        
        # compute training accuracy
        flattened_targets = labels.view(-1) # shape (batch_size * seq_len,)
        active_logits = tr_logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
        flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
        
        # only compute accuracy at active labels
        active_accuracy = labels.view(-1) != -100 # shape (batch_size, seq_len)
        #active_labels = torch.where(active_accuracy, labels.view(-1), torch.tensor(-100).type_as(labels))
        
        #-100을 거르는 용도!
        labels = torch.masked_select(flattened_targets, active_accuracy)
        predictions = torch.masked_select(flattened_predictions, active_accuracy)
        
        tr_labels.extend(labels)
        tr_preds.extend(predictions)

        tmp_tr_accuracy = accuracy_score(labels.cpu().numpy(), predictions.cpu().numpy())
        tr_accuracy += tmp_tr_accuracy
    
        # gradient clipping
        torch.nn.utils.clip_grad_norm_(
            parameters=model.parameters(), max_norm=config['max_grad_norm']
        )
        
        # backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    epoch_loss = tr_loss / nb_tr_steps
    tr_accuracy = tr_accuracy / nb_tr_steps
    print(f"Training loss epoch: {epoch_loss}")
    print(f"Training accuracy epoch: {tr_accuracy}")

In [ ]:
for epoch in range(config['epochs']):
    print(f"Training epoch: {epoch + 1}")
    train(epoch)

In [ ]:
def valid(model, testing_loader):
    # put model in evaluation mode
    model.eval()
    
    eval_loss, eval_accuracy = 0, 0
    nb_eval_examples, nb_eval_steps = 0, 0
    eval_preds, eval_labels = [], []
    
    with torch.no_grad():
        for idx, batch in enumerate(testing_loader):
            
            ids = batch['input_ids'].to(device, dtype = torch.long)
            mask = batch['attention_mask'].to(device, dtype = torch.long)
            labels = batch['labels'].to(device, dtype = torch.long)
            
            loss, eval_logits = model(input_ids=ids, attention_mask=mask, labels=labels, return_dict=False)
            
            eval_loss += loss.item()

            nb_eval_steps += 1
            nb_eval_examples += labels.size(0)
        
            if idx % 100==0:
                loss_step = eval_loss/nb_eval_steps
                print(f"Validation loss per 100 evaluation steps: {loss_step}")
            # compute evaluation accuracy
            flattened_targets = labels.view(-1) # shape (batch_size * seq_len,)
            active_logits = eval_logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
            flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
            
            # only compute accuracy at active labels
            active_accuracy = labels.view(-1) != -100 # shape (batch_size, seq_len)
        
            labels = torch.masked_select(flattened_targets, active_accuracy)
            predictions = torch.masked_select(flattened_predictions, active_accuracy)
            
            eval_labels.extend(labels)
            eval_preds.extend(predictions)
            
            tmp_eval_accuracy = accuracy_score(labels.cpu().numpy(), predictions.cpu().numpy())
            eval_accuracy += tmp_eval_accuracy

    labels = [ids_to_labels[id.item()] for id in eval_labels]
    predictions = [ids_to_labels[id.item()] for id in eval_preds]
    
    eval_loss = eval_loss / nb_eval_steps
    eval_accuracy = eval_accuracy / nb_eval_steps
    print(f"Validation Loss: {eval_loss}")
    print(f"Validation Accuracy: {eval_accuracy}")

    return labels, predictions

In [ ]:
labels, predictions = valid(model, testing_loader)

In [ ]:
sentence = "@HuggingFace is a company based in New York, but is also has employees working in Paris"
model.eval()
def inference(sentence):
    inputs = tokenizer(sentence,
#                         is_split_into_words=True, 
                        return_offsets_mapping=True, 
                        padding='max_length', 
                        truncation=True, 
                        max_length=config['max_length'],
                        return_tensors="pt")

    # move to gpu
    ids = inputs["input_ids"].to(device)
    mask = inputs["attention_mask"].to(device)
    # forward pass
    outputs = model(ids, attention_mask=mask, return_dict=False)
    logits = outputs[0]

    active_logits = logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
    flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size*seq_len,) - predictions at the token level

    tokens = tokenizer.convert_ids_to_tokens(ids.squeeze().tolist())
    token_predictions = [ids_to_labels[i] for i in flattened_predictions.cpu().numpy()]
    wp_preds = list(zip(tokens, token_predictions)) # list of tuples. Each tuple = (wordpiece, prediction)

    prediction = []
    out_str = []
    off_list = inputs["offset_mapping"].squeeze().tolist()
    for idx, mapping in enumerate(off_list):
#         print(mapping, token_pred[1], token_pred[0],"####")

#         only predictions on first word pieces are important
        if mapping[0] != 0 and mapping[0] != off_list[idx-1][1]:
#             print(mapping, token_pred[1], token_pred[0])
            prediction.append(wp_preds[idx][1])
            out_str.append(wp_preds[idx][0])
        else:
            if idx == 1:
                prediction.append(wp_preds[idx][1])
                out_str.append(wp_preds[idx][0])
            continue
    return prediction, out_str

In [ ]:
test_texts.head()

In [ ]:
y_pred = []

for i, t in enumerate(test_texts['text'].tolist()):
    o,o_t = inference(t)
    y_pred.append(o)
    l = train_text_df['entities'][i]

In [ ]:
final_preds = []
import pdb
for i in tqdm(range(len(test_texts))):
#     pdb.set_trace()
    idx = test_texts.id.values[i]
#     pred = ['']*len(test_texts[i])

#     for j in range(len(y_pred[i])):
#         if words[i][j] != None:
#             pred[words[i][j]] = labels[y_pred[i][j]]

    pred = [x.replace('B-','').replace('I-','') for x in y_pred[i]]
#     print(pred)
    preds = []
    j = 0
    while j < len(pred):
        cls = pred[j]
#         pdb.set_trace()
        if cls == 'O':
            j += 1
        end = j + 1
        while end < len(pred) and pred[end] == cls:
            end += 1
            
        if cls != 'O' and cls != '' and end - j > 10:
            final_preds.append((idx, cls, ' '.join(map(str, list(range(j, end))))))
        
        j = end
        
print(final_preds[1])

In [ ]:
len(final_preds)

In [ ]:
test_df = pd.read_csv('./sample_submission.csv')
test_df

In [ ]:
sub = pd.DataFrame(final_preds)
sub.columns = test_df.columns

In [ ]:
sub.head()

In [ ]:
sub.to_csv("submission.csv", index=False)